# Federated PyTorch MNIST Tutorial

In [1]:
#Install dependencies if not already installed
!pip install torch torchvision

You should consider upgrading via the '/home/itrushkin/.virtualenvs/openfl/bin/python -m pip install --upgrade pip' command.


In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms
import openfl.native as fx
from openfl.federated import FederatedModel,FederatedDataSet
from fedcurv import FedCurv

torch.manual_seed(0)
np.random.seed(0)

After importing the required packages, the next step is setting up our openfl workspace. To do this, simply run the `fx.init()` command as follows:

In [3]:
#Setup default workspace, logging, etc.
fx.init('torch_cnn_mnist')

Creating Workspace Directories
Creating Workspace Templates

New workspace directory structure:
workspace
├── plan
│   ├── defaults
│   │   ├── data_loader.yaml
│   │   ├── network.yaml
│   │   ├── tasks_keras.yaml
│   │   ├── collaborator.yaml
│   │   ├── assigner.yaml
│   │   ├── task_runner.yaml
│   │   ├── defaults
│   │   ├── tasks_tensorflow.yaml
│   │   ├── compression_pipeline.yaml
│   │   ├── aggregator.yaml
│   │   ├── tasks_torch.yaml
│   │   └── tasks_fast_estimator.yaml
│   ├── plan.yaml
│   ├── data.yaml
│   └── cols.yaml
├── logs
│   ├── 2021-09-10T16:56:52.795814
│   │   └── events.out.tfevents.1631282212.nnlicv611
│   ├── 2021-09-10T21:49:00.280909
│   │   └── events.out.tfevents.1631299740.nnlicv611
│   ├── 2021-09-10T18:10:48.457623
│   │   └── events.out.tfevents.1631286648.nnlicv611
│   ├── 2021-09-10T16:04:13.204925
│   │   └── events.out.tfevents.1631279053.nnlicv611
│   ├── 2021-09-10T17:21:03.288731
│   │   └── events.out.tfevents.1631283663.nnlicv611
│   ├── 2

Now we are ready to define our dataset and model to perform federated learning on. The dataset should be composed of a numpy arrayWe start with a simple fully connected model that is trained on the MNIST dataset. 

In [4]:
def one_hot(labels, classes):
    return np.eye(classes)[labels]

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)

train_images,train_labels = trainset.train_data, np.array(trainset.train_labels)
train_images = torch.from_numpy(np.expand_dims(train_images, axis=1)).float()
train_labels = one_hot(train_labels,10)

validset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)

valid_images,valid_labels = validset.test_data, np.array(validset.test_labels)
valid_images = torch.from_numpy(np.expand_dims(valid_images, axis=1)).float()
valid_labels = one_hot(valid_labels,10)

/home/itrushkin/.virtualenvs/openfl/lib/python3.8/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)
/home/itrushkin/.virtualenvs/openfl/lib/python3.8/site-packages/torchvision/datasets/mnist.py:62: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/home/itrushkin/.virtualenvs/openfl/lib/python3.8/site-packages/torchvision/datasets/mnist.py:52: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels ha

In [5]:
feature_shape = train_images.shape[1]
classes       = 10

fl_data = FederatedDataSet(train_images,train_labels,valid_images,valid_labels,batch_size=32,num_classes=classes)

class Net(FedCurv):
    def __init__(self):
        super().__init__(importance=1e-1) # we need to call base constructor after parameters definition to create fisher placeholders
        self.conv1 = nn.Conv2d(1, 16, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3)
        self.fc1 = nn.Linear(32 * 5 * 5, 32)
        self.fc2 = nn.Linear(32, 84)
        self.fc3 = nn.Linear(84, 10)
        self.register_fisher_parameters()
        

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0),-1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    
optimizer = lambda x: optim.Adam(x, lr=1e-4)

def cross_entropy(output, target):
    """Binary cross-entropy metric
    """
    return F.cross_entropy(input=output,target=torch.argmax(target, dim=1))

In [6]:

#Create a federated model using the pytorch class, lambda optimizer function, and loss function
fl_model = FederatedModel(build_model=Net,optimizer=optimizer,loss_fn=cross_entropy,data_loader=fl_data)

The `FederatedModel` object is a wrapper around your Keras, Tensorflow or PyTorch model that makes it compatible with openfl. It provides built in federated training and validation functions that we will see used below. Using it's `setup` function, collaborator models and datasets can be automatically defined for the experiment. 

In [7]:
collaborator_models = fl_model.setup(num_collaborators=10)
collaborators = {str(i): collaborator_models[i] for i in range(10)}#, 'three':collaborator_models[2]}

In [8]:
#Original MNIST dataset
print(f'Original training data size: {len(train_images)}')
print(f'Original validation data size: {len(valid_images)}\n')

#Collaborator one's data
print(f'Collaborator one\'s training data size: {len(collaborator_models[0].data_loader.X_train)}')
print(f'Collaborator one\'s validation data size: {len(collaborator_models[0].data_loader.X_valid)}\n')

#Collaborator two's data
print(f'Collaborator two\'s training data size: {len(collaborator_models[1].data_loader.X_train)}')
print(f'Collaborator two\'s validation data size: {len(collaborator_models[1].data_loader.X_valid)}\n')

#Collaborator three's data
#print(f'Collaborator three\'s training data size: {len(collaborator_models[2].data_loader.X_train)}')
#print(f'Collaborator three\'s validation data size: {len(collaborator_models[2].data_loader.X_valid)}')

Original training data size: 60000
Original validation data size: 10000

Collaborator one's training data size: 6000
Collaborator one's validation data size: 1000

Collaborator two's training data size: 6000
Collaborator two's validation data size: 1000



We can see the current plan values by running the `fx.get_plan()` function

In [9]:
 #Get the current values of the plan. Each of these can be overridden
print(fx.get_plan())

{
    "aggregator.settings.best_state_path": "save/torch_cnn_mnist_best.pbuf",
    "aggregator.settings.db_store_rounds": 1,
    "aggregator.settings.init_state_path": "save/torch_cnn_mnist_init.pbuf",
    "aggregator.settings.last_state_path": "save/torch_cnn_mnist_last.pbuf",
    "aggregator.settings.log_metric_callback": {
        "template": "src.mnist_utils.write_metric"
    },
    "aggregator.settings.rounds_to_train": 10,
    "aggregator.template": "openfl.component.Aggregator",
    "assigner.settings.task_groups": [
        {
            "name": "train_and_validate",
            "percentage": 1.0,
            "tasks": [
                "aggregated_model_validation",
                "train",
                "locally_tuned_model_validation"
            ]
        }
    ],
    "assigner.template": "openfl.component.RandomGroupedAssigner",
    "collaborator.settings.db_store_rounds": 1,
    "collaborator.settings.delta_updates": false,
    "collaborator.settings.opt_treatment": "RES

Now we are ready to run our experiment. If we want to pass in custom plan settings, we can easily do that with the `override_config` parameter

In [10]:
#Run experiment, return trained FederatedModel
final_fl_model = fx.run_experiment(collaborators,
                                   {
                                       'aggregator.settings.rounds_to_train':5,
                                       'aggregator.settings.db_store_rounds':5
                                   })

/home/itrushkin/.virtualenvs/openfl/lib/python3.8/site-packages/openfl/federated/task/runner_pt.py:106: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data, target = pt.tensor(data).to(self.device), pt.tensor(
/home/itrushkin/.virtualenvs/openfl/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


/home/itrushkin/repos/openfl/openfl-tutorials/FedCurv/fedcurv.py:71: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data, target = torch.tensor(data).to(self.device), torch.tensor(target).to(self.device)
/home/itrushkin/repos/openfl/openfl-tutorials/FedCurv/fedcurv.py:84: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  add_param(self.model, f'{n}_fisher', torch.tensor(m))
